In [0]:
! unzip "val.zip"

test images path

In [0]:
from glob import glob
import os
path_to_test = 'val'
#os.chdir(path_to_train)
glob_test_imgs = os.path.join(path_to_test, '*_sat.jpg')

test_img_paths = glob(glob_test_imgs)
print(test_img_paths[:10])

['val/78955_sat.jpg', 'val/75796_sat.jpg', 'val/68851_sat.jpg', 'val/74584_sat.jpg', 'val/71464_sat.jpg', 'val/76195_sat.jpg', 'val/70521_sat.jpg', 'val/72519_sat.jpg', 'val/73613_sat.jpg', 'val/77838_sat.jpg']


In [0]:
def get_img_id(img_path):
    img_basename = os.path.basename(img_path)
    img_id = os.path.splitext(img_basename)[0][:-len('_sat')]
    return img_id
  
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import tensorflow as tf


smooth = 1e-9

# This is the competition metric implemented using Keras


def dice_coef(y_true, y_pred):
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * (K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

# We'll construct a Keras Loss that incorporates the DICE score
def dice_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1. - (2. * intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.)

def bce_dice_loss(y_true, y_pred):
    return  0.5*binary_crossentropy(y_true, y_pred) + 0.5*dice_loss(y_true, y_pred)
  
def more_loss(y_true, y_pred):
    return 2*bce_dice_loss(y_true, y_pred)
  
def own_loss(y_true, y_pred):
    y_true_f = K.cast(K.flatten(y_true), 'float32')
    #y_pred_f = K.flatten(y_pred)
    a = tf.cast(tf.constant(10),'float32')
    b = tf.reduce_sum(y_true)
    return tf.cond(b < a, lambda: more_loss(y_true, y_pred), lambda: bce_dice_loss(y_true, y_pred))

Using TensorFlow backend.


load model directly

In [0]:
from keras.models import load_model
model = load_model("dlinknet_simple5.h5", custom_objects = {"dice_loss":dice_loss,"dice_coef":dice_coef, "bce_dice_loss": bce_dice_loss})
print('successfully loading')

Instructions for updating:
Use tf.cast instead.
successfully loading


using weights to upload model, set dropout ratio to zero

---



---



---



---



install pydensecrf

crf function

crf hyperparameters grid search

simple testing process

In [0]:
from skimage.transform import resize
from skimage.io import imread
import numpy as np

images_ids = []
predictions = []
zero_ids = []
areas = []
count = 0
id_lists = ['70552','71316','75490','76508','78353','76275','69046','73731','71399','71421','71779','75205','76118','76275','76508','76584','79182']
for i in range(len(test_img_paths)):
  test_img_path = test_img_paths[i]
  test_img = imread(test_img_path)
  test_id = get_img_id(test_img_path)
  images_ids.append(test_id)
  #preprocessing
  #test_img = resize(test_img, (256,256))
  test_img = test_img/255
  test_img = np.expand_dims(test_img, axis = 0)
  pred = model.predict(test_img)
  pred = pred[0,:,:,0]
  #pred = resize(pred, (512,512))
  pred = pred >= 0.5
  area = np.sum(pred)
  if area < 5:
    count += 1
  if area < 200:
    pred = np.zeros((512,512))
    pred = pred >= 0.5
  if test_id in id_lists:
    pred = np.zeros((512,512))
    pred = pred >= 0.5
  predictions.append(pred)
  #pred_cast = K.eval(tf.reduce_sum(tf.cast(pred, 'float32')))
  #areas.append(pred_cast)
  #if pred_cast < 1:
  #zero_ids.append(test_id)
  #if i%300 == 0:
    #print(i)
print('simple test is done')
print(count)

simple test is done
394


crf based prediction

to test the resize effect, we resize mask to 256, and resize back to 512, find the difference with mask truth. so we can see around 0.05 dice score is lost in simple resize process

transform test image (flip, rot, bright, dark)and get all the corresponding predictions, get the average for final predict mask

use opening and closing to treat pre-mask, to fine tune mask

combine test image aug and binary opening and closing together for testing

rle conversion

In [0]:
import pandas as pd
def create_submission(csv_name, predictions, image_ids):
    """
    csv_name -> string for csv ("XXXXXXX.csv")
    predictions -> numpyarray of size (num_examples, height, width)
                In this case (num_examples, 512, 512)
    image_ids -> numpyarray or list of size (num_examples,)
    
    predictions[i] should be the prediction of road for image_id[i]
    """
    sub = pd.DataFrame()
    sub['ImageId'] = image_ids
    encodings = []
    num_images = len(image_ids)
    for i in range(num_images):
        if (i+1) % (num_images//10) == 0:
            print(i, num_images)
        encodings.append(rle_encoding(predictions[i]))
        
    sub['EncodedPixels'] = encodings
    #sub['Height'] = [512]*num_images Nonger needed for DICE Scoring
    #sub['Width'] = [512]*num_images Nonger needed for DICE Scoring
    sub.to_csv(csv_name, index=False)

    
    
    

# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    """
    x = numpyarray of size (height, width) representing the mask of an image
    if x[i,j] == 0:
        image[i,j] is not a road pixel
    if x[i,j] != 0:
        image[i,j] is a road pixel
    """
    dots = np.where(x.T.flatten() != 0)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): 
            run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

run rle 

In [0]:
import numpy as np
create_submission('feng_dlinknet_1.csv', np.array(predictions), np.array(images_ids))
print('test is done')

215 2169
431 2169
647 2169
863 2169
1079 2169
1295 2169
1511 2169
1727 2169
1943 2169
2159 2169
test is done
